# Steps to generate recommendations
1. Run StartDaemons.ipynb to setup the servers
2. Put your username and source in the cell below. For example, setting username="Fro116" and source="mal" will generate recommendations for the user "Fro116" on the site "myanimelist.net"
3. Run the script to generate an html page of recommendations

In [ ]:
username = ""
source = ""

In [ ]:
import os
import shutil
import subprocess

In [ ]:
sources = [
    "mal",
    "anilist",
    "kitsu",
    "animeplanet",
]
assert username, "Must provide a username"
assert source in sources, "Must provide a list source"
if "@" in username:
    fields = username.split("@")
    assert len(fields) == 2 and fields[0] == source

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
if "@" in username:
    rec_dir = os.path.join(data_path, *username.split("@"))
else:
    rec_dir = os.path.join(data_path, source, str(username))
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
source_to_port = {"mal": 3000, "anilist": 3001, "kitsu": 3002, "animeplanet": 3003}
python_script_to_port = {
    "Transformer": 3004,
    "BagOfWords": 3005,
}
julia_script_to_port = {
    "CompressSplits.jl": 3006,
    "Baseline.jl": 3007,
    "BagOfWords.jl": 3008,
    "Nondirectional.jl": 3009,
    "Transformer.jl": 3010,
    "Ensemble.jl": 3011,
    "Recommendations.jl": 3012,
}

In [ ]:
def spawn_parallel(commands):
    procs = []
    for cmd in commands:
        procs.append(
            subprocess.Popen(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        )
    for p in procs:
        p.wait()


def create_julia_command(x):
    port = julia_script_to_port[x]
    url = f"http://localhost:{port}/query?username={username}&source={source}"
    if x in ["Transformer.jl", "BagOfWords.jl"]:
        base = x.split(".")[0]
        url += f"&modelport={python_script_to_port[base]}"
    return ["curl", url]

# Query servers

In [ ]:
%%time
spawn_parallel(
    [
        [
            "curl",
            f"http://localhost:{source_to_port[source]}/query?username={username}&medium={medium}",
        ]
        for medium in ["manga", "anime"]
    ]
)

In [ ]:
username = str(username)
if "@" in username:
    username = username.split("@")[1]

In [ ]:
%%time
spawn_parallel([create_julia_command(x) for x in ["CompressSplits.jl"]])
spawn_parallel([create_julia_command(x) for x in ["Baseline.jl"]])
spawn_parallel(
    [
        create_julia_command(x)
        for x in ["BagOfWords.jl", "Nondirectional.jl", "Transformer.jl"]
    ]
)
spawn_parallel([create_julia_command(x) for x in ["Ensemble.jl"]])
spawn_parallel([create_julia_command(x) for x in ["Recommendations.jl"]])